# Homework 08 - Implementation
### Scott Chase Waggener

# Problem 1
# Problem 2 
# Problem 3 
# Problem 4

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import InputLayer
import tensorflow_hub as hub
import datetime, os
import shutil
%reload_ext tensorboard.notebook
print('Requires Tensorflow 2.0 Alpha')
print('Tensorflow: %s' % tf.__version__)

Requires Tensorflow 2.0 Alpha
Tensorflow: 2.0.0-dev20190326


### Defining the Model

We can use high level APIs in Tensorflow to build one of several well known models. Here we set `weights=None` because our input shape does not match that of the trained model.

In [2]:
input_shape = (512, 1024, 3)

# Set weights=None or input_shape must match trained shape
# Set include_top=False to generate a headless model
model = tf.keras.applications.MobileNetV2(
    input_shape=input_shape,
    weights=None  
)

We can then compile the model with generic parameters and get a summary of the layers involved.

In [3]:
# Give any optimizer/loss for successful compilation
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.summary()

Model: "mobilenetv2_1.00_512"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 1024, 3 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 513, 1025, 3) 0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 256, 512, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 256, 512, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

### Visualizing the Graph

With Tensorflow 2.0 we can visualize the computational graph directly in the notebook with Tensorboard. To do so, we provide a trivial input, a Tensorboard callback, and call `model.predict()` on the trivial input to compile the graph. When actually training a model this happens inside `model.fit()` without needing to give a trivial input.

**Note:** Models do not automatically disappear from the computational graph. Tensorflow 2.0 was supposed to handle garbage collection of graph objects but perhaps this doesnt work in a REPL environment? If you have many duplicates of a model present in the graph, launching Tensorboard may freeze your browser for several seconds. This can be resolved by restarting the kernel (or finding a Tensorflow call to reset the default graph, or manually specifying a graph scope). 

In [9]:
from tensorflow.keras.callbacks import TensorBoard

tensorboard = TensorBoard(profile_batch=1)

# Create trivial input for predict() call
inputs = tf.zeros(
    shape=(1,) + input_shape,
    dtype=tf.float32
)
outputs = model(inputs)
tensorboard.set_model(model)


# Clear Tensorboard logs and launch
shutil.rmtree('./logs', ignore_errors=True)
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 17584), started -1 day, 23:02:44 ago. (Use '!kill 17584' to kill it.)

In [8]:

# Run the model on single input to trigger graph build
_ = model.predict(
    inputs,
    steps=1,
    batch_size=1,
    callbacks=[tensorboard]
)

This takes us through the graph portion of the problem. Through Tensorboard or `model.summary()` we have a mechanism of exporting layers and their dimensions to a text file or other location. In fact, with Tensorboard and the associated log files (which are deleted in the above code but can always be retained) we can restore a graph at a later time.